In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
from torchvision import models

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:

transform_test = transforms.Compose([
    transforms.Resize((224,224)), 
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])

In [4]:
num_classes = 25
classes=['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy',
         'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___healthy','Corn_(maize)___Northern_Leaf_Blight',
         'Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
         'Potato___Early_blight','Potato___healthy','Potato___Late_blight','Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight',
         'Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot',
         'Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus']

In [5]:
file='resnet152.pth'

In [6]:
loaded_model=models.resnet152(pretrained=True)

In [7]:
for param in loaded_model.parameters():
    param.requires_grad=False

In [8]:
in_features=loaded_model.fc.in_features
loaded_model.fc=nn.Sequential(nn.Linear(in_features,64),
                        torch.nn.Dropout(0.25),
                        nn.LeakyReLU(),
                        nn.Linear(64,25)
                       )

In [9]:
loaded_model.load_state_dict(torch.load(file))
loaded_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
loaded_model=loaded_model.to(device)

In [65]:
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from PIL import ImageTk, Image
filename=None
img=None

def upload():
    global filename,img,root,img2,uploaded 
    filename=filedialog.askopenfilename(filetypes=[("Image File",'.jpg')])
    img=Image.open(filename)
    img=img.resize((400,250))
    img = ImageTk.PhotoImage(img)
    label_1=Label(root,image=img)
    label_1.place(x=50,y=15,width="400",height="250")
fnt=('Arial',12,'bold')   

def predict():
    global filename
    from PIL import Image
    import torch.nn.functional as F
    img_path=filename
    img1=Image.open(img_path)
    img1_trans=transform_test(img1)
    img1_trans_reshaped=img1_trans.view(-1,3,224,224)
    out=loaded_model(img1_trans_reshaped.to(device))
    out_softmax=F.softmax(out,1)
    max_val,pred=torch.max(out_softmax.data,1)
    classes=['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy',
         'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot','Corn_(maize)___Common_rust_','Corn_(maize)___healthy','Corn_(maize)___Northern_Leaf_Blight',
         'Grape___Black_rot','Grape___Esca_(Black_Measles)','Grape___healthy','Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
         'Potato___Early_blight','Potato___healthy','Potato___Late_blight','Tomato___Bacterial_spot','Tomato___Early_blight','Tomato___healthy','Tomato___Late_blight',
         'Tomato___Leaf_Mold','Tomato___Septoria_leaf_spot','Tomato___Spider_mites Two-spotted_spider_mite','Tomato___Target_Spot',
         'Tomato___Tomato_mosaic_virus','Tomato___Tomato_Yellow_Leaf_Curl_Virus']
    prediction=classes[pred.data.item()]
    txt=prediction
    
    l=Label(root,font=fnt,text=txt,fg='Gray24',bg='green',relief=SUNKEN)
    l.place(x=50,y=450,width=400,height=45)

root = Tk()
root.configure(background='gray')
root.geometry("500x500")
f=Frame(root,width=500,height=500,bg='gray')    
f.pack()
x=Button(root,cursor='hand2',text="   Fotoğrafı yükle   ",bg="gold",fg="black",width=15,height=1,font=('Arial',13,'bold'),relief=RAISED,command=upload)
x.place(x=175,y=300)
y=Button(root,cursor='hand2',text="   Tahmin Et   ",bg="gold",fg="black",width=15,height=1,font=('Arial',13,'bold'),relief=RAISED,command=predict)
y.place(x=175,y=400)
label1=Label(root,cursor='xterm',bg='yellow',relief=SUNKEN,text='Input Image')
label1.place(x=50,y=15,width="400",height="250")
root.mainloop()